In [1]:
import sqlite3
import pandas as pd
from sqlalchemy import create_engine
import numpy as np
import yfinance as yf
from datetime import datetime
import os


def LoadData():
    # Maak connectie met de database en geef de locaties aan van de input bestanden
    conn = sqlite3.connect('DatabaseVB.db')
    posdirectory = './Input/Posrecon'
    tradedirectory = './Input/Traderecon'
    
    # Loop over de input bestanden en laad ze in de database
    for file in os.listdir(posdirectory):
        df = pd.read_csv(posdirectory+'/'+file)
        df.to_sql('Posrecon', if_exists = "replace", con = conn)
    
    for file in os.listdir(tradedirectory):
        df = pd.read_csv(tradedirectory+'/'+file)
        df.to_sql('Traderecon', if_exists = "replace", con = conn)


# def FileUploaderPosrecon(inputdata):
#     conn = sqlite3.connect('DatabaseVB.db')  
#     df = pd.read_csv(inputdata)
#     df.to_sql('Posrecon', if_exists = "replace", con = conn)

# def FileUploaderTraderecon(inputdata):
#     conn = sqlite3.connect('DatabaseVB.db')
#     df = pd.read_csv(inputdata)
#     df.to_sql('Traderecon', if_exists = "replace", con = conn)

def GetRendement(x):
    engine = create_engine('sqlite:///DatabaseVB.db')
    
    # Lees Posrecon table uit de database
    df_posrecon = pd.read_sql(f'''SELECT "Datum", ROUND(sum("Waarde EUR"),2) as "Eind Waarde" 
                      FROM "Posrecon" where "RekNr" = "{x}" group by "Datum" order by "Datum"''', con = engine).set_index('Datum')

    ### LEES UIT DE DATABASE DE SOM VAN DE ONTTREKKINGEN / OVERBOEKINGEN / LICHTINGEN / STORTINGEN VOOR REKNR X
    df_onttrekking = pd.read_sql(f''' Select Datum, sum("Aantal") as "Onttrekkingen" from Traderecon
                       where RekNr = "{x}" and "Unnamed: 34" = 5025 OR "Unnamed: 34" = 5000 group by "Datum" ''', con = engine).set_index('Datum')

    df_stortingen = pd.read_sql(f''' Select Datum, sum("Aantal") as "Stortingen" from Traderecon
                       where RekNr = "{x}"  and "Unnamed: 34" = 5026 group by "Datum" ''', con = engine).set_index('Datum')

    df_lichtingen = pd.read_sql(f''' Select Datum, sum("Aantal") as "Lichtingen" from Traderecon
                        where RekNr = "{x}" and "Type" = "L" group by "Datum" ''', con = engine).set_index('Datum')

    df_deponeringen = pd.read_sql(f''' Select Datum, sum("Aantal") as "Deponeringen" from Traderecon
                        where RekNr = "{x}" and "Type" = "D" group by "Datum" ''', con = engine).set_index('Datum')
    
    # Concat de 4 dataframes uit de Traderecon query in 1 dataframe en merge deze met de Posrecon dataframe
    traderecon_data = [df_onttrekking, df_stortingen, df_lichtingen, df_deponeringen]
    df_tot_tr = pd.concat(traderecon_data)
    df_final = df_posrecon.merge(df_tot_tr, on='Datum', how='left')
    
    ### Vervang NAN van de mutaties met 0
    traderecon_columns = ['Onttrekkingen','Stortingen', 'Lichtingen','Deponeringen']
    df_final[traderecon_columns] = df_final[traderecon_columns].fillna(0.0)
    
    ### MAAK KOLOM ACTUELE RENDEMENT EN BEREKEN RENDEMENT VAN WAARDE PORTEFEUILLE EN ONTTREKKINGEN / STORTINGEN
    df_final['Start Waarde'] = df_final["Eind Waarde"].shift(1)
    df_final['Dag Rendement'] = ((df_final['Eind Waarde'] - df_final['Start Waarde'] - df_final['Stortingen'] - df_final['Deponeringen'] + df_final['Onttrekkingen'] + df_final['Lichtingen'] ) ) / (df_final['Start Waarde'] + df_final['Stortingen'] - df_final['Onttrekkingen']).round(5)
    df_final['Portfolio Cumulatief Rendement'] = (1 + df_final['Dag Rendement']).cumprod()
    #df_final['Eind Waarde'] =  pd.to_numeric(df_final['Eind Waarde'], downcast = 'float')
    columns = ['Start Waarde','Stortingen','Deponeringen', 'Onttrekkingen', 'Lichtingen', 'Eind Waarde', 'Dag Rendement', 'Portfolio Cumulatief Rendement']
    
    return df_final[columns]

periode = {
    'Q1':
    {'start':'2020-01-02',
    'end':'2020-01-31'},
    'Q2':
    {'start':'2020-02-03',
    'end':'2020-02-28'},
    'Q3':
    {'start':'2020-02-03',
    'end':'2020-03-31'},
    'Q4':
    {'start':'2020-10-01',
    'end':'2020-12-31'},
    'YTD':
    {'start':'2020-01-01',
    'end':datetime.today().strftime('%Y-%m-%d')}
}

# Overview portefeuille Ontwikkeling
def GetOverview(data, kwartaals): 
    startwaarde, stortingen, deponeringen, onttrekkingen, lichtingen,eindwaarde = [],[],[],[],[],[]
    for kwartaal in kwartaals:
        startwaarde.append(data.loc[periode[kwartaal]['start'],['Start Waarde']][0])
        stortingen.append((data.loc[periode[kwartaal]['start']:periode[kwartaal]['end'],['Stortingen']]).sum()[0])
        deponeringen.append((data.loc[periode[kwartaal]['start']:periode[kwartaal]['end'],['Deponeringen']]).sum()[0])
        onttrekkingen.append((data.loc[periode[kwartaal]['start']:periode[kwartaal]['end'],['Onttrekkingen']]).sum()[0])
        lichtingen.append((data.loc[periode[kwartaal]['start']:periode[kwartaal]['end'],['Lichtingen']]).sum()[0])
        eindwaarde.append(data.loc[periode[kwartaal]['end'],['Eind Waarde']][0])
    overview = list(zip(startwaarde, stortingen, deponeringen, onttrekkingen, lichtingen, eindwaarde))
    
    df = pd.DataFrame(overview, 
           columns=["Start Waarde","Stortingen","Deponeringen","Onttrekkingen","Lichtingen","Eind Waarde"], index = kwartaals)
    df['Abs Rendement'] = df['Eind Waarde'] - df['Start Waarde'] - df['Stortingen'] - df['Deponeringen'] + df['Onttrekkingen'] + df['Lichtingen']
    df['Rendement'] = (df['Eind Waarde'] - df['Start Waarde']) / df['Start Waarde']
    return df

# Full Benchmark data
def getBenchmarkData(bench):
    conn = sqlite3.connect('DatabaseVB.db')
    engine = create_engine('sqlite:///DatabaseVB.db')
    ticker = yf.Ticker(bench)

    df_benchmark = ticker.history(period = 'max')
    df_benchmark.reset_index(inplace = True)
    df_benchmark.rename(columns = {'Date':'Datum', 'Close': f'{bench} Eind Waarde'}, inplace = True)
    df_benchmark.to_sql(f'{bench}', if_exists = 'replace', con = conn)

    df = pd.read_sql(f'''
        SELECT substr(Datum, 1, 10) as "Datum", "{bench} Eind Waarde" FROM "{bench}"
    ''', con = engine).set_index("Datum")
    return df

#  Benchmark Ontwikkeling
def getPerf(data, kwartaals, bench):
    kwart, startwaarde, eindwaarde = [], [], []
    for kwartaal in kwartaals:
        kwart.append(kwartaal)
        startwaarde.append(data.loc[periode[kwartaal]['start']][0])
        eindwaarde.append(data.loc[periode[kwartaal]['end']][0])

        overview = list(zip(kwart, startwaarde, eindwaarde))

        df = pd.DataFrame(overview, columns=['Kwartaal','Start Waarde','Eind Waarde'],
                         index = kwart)
        
        df['Benchmark Performance'] = (df['Eind Waarde'] - df['Start Waarde']) / df['Start Waarde']     
    return df


In [2]:
LoadData()

c:\users\ismet\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\generic.py:2602: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [3]:
df1 = GetRendement(5)

In [6]:
aex = getBenchmarkData("^AEX")

c:\users\ismet\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\generic.py:2602: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [32]:
start_date = '2020-01-06'
end_date = '2020-02-12'

In [11]:
# Handmatig kiezen van start- en einddatum voor de portefeuille ontwikkeling
def ZoekPortfOntwikkeling(data, start_datum, eind_datum):
    df = data.loc[start_datum:eind_datum]
    portf_startwaarde = df.loc[start_datum,['Start Waarde']][0]
    portf_stortingen = df.loc[start_datum:eind_datum,['Stortingen']].sum()[0]
    portf_deponeringen = df.loc[start_datum:eind_datum,['Deponeringen']].sum()[0]
    portf_onttrekkingen = df.loc[start_datum:eind_datum,['Onttrekkingen']].sum()[0]
    portf_lichtingen = df.loc[start_datum:eind_datum,['Lichtingen']].sum()[0]
    portf_eindwaarde = df.loc[eind_datum,['Eind Waarde']][0]

    overview = [portf_startwaarde, portf_stortingen, portf_deponeringen, portf_onttrekkingen, portf_lichtingen, 
               portf_eindwaarde]
    return pd.DataFrame([overview], columns = ['Start Waarde','Stortingen','Deponeringen', 'Onttrekkingen', 'Lichtingen', 'Eind Waarde'])

In [33]:
test_d = ZoekPortfOntwikkeling(df1,start_date,end_date)

In [34]:
aex_d = getBenchmarkData('^AEX')

c:\users\ismet\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\generic.py:2602: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [44]:
def ZoekBenchmarkOntwikkeling(data,start_date, end_date):
    new_benchmark_df = aex_d[start_date:end_date]
    bench_sw = new_benchmark_df.loc[start_date][0]
    bench_ew = new_benchmark_df.loc[end_date][0]
    overview = [bench_sw, bench_ew]
    df = pd.DataFrame([overview], columns =['Start Waarde', 'Eind Waarde'])
    df['Rendement'] = bench_ew - bench_sw / bench_sw
    return df

In [45]:
ZoekBenchmarkOntwikkeling(df1, start_date, end_date)

,Start Waarde,Eind Waarde,Rendement
0,607.24,629.03,628.03


In [10]:
def LoadData():
    # Maak connectie met de database en geef de locaties aan van de input bestanden
    conn = sqlite3.connect('DatabaseVB.db')
    posdirectory = './Input/Posrecon'
    tradedirectory = './Input/Traderecon'
    
    # Loop over de input bestanden en laad ze in de database
    for file in os.listdir(posdirectory):
        df = pd.read_csv(posdirectory+'/'+file)
        df.to_sql('Posrecon', if_exists = "replace", con = conn)
    
    for file in os.listdir(tradedirectory):
        df = pd.read_csv(tradedirectory+'/'+file)
        df.to_sql('Traderecon', if_exists = "replace", con = conn)

In [11]:
LoadData()

c:\users\ismet\appdata\local\programs\python\python38-32\lib\site-packages\pandas\core\generic.py:2602: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  sql.to_sql(


In [12]:
for file in os.listdir(posdirectory):
        print(posdirectory+'/'+file)

./Input/Posrecon/1.csv
./Input/Posrecon/Posrecon.csv
